# Setup

## Load Modules

In [1]:
# Standard library
import os.path

# External modules
from neuron import h
import bluepyopt.ephys as ephys

# Our modules
from bgcellmodels.common import electrotonic, treeutils
from bgcellmodels.morphology import morph_ni

from bgcellmodels.models.STN.GilliesWillshaw import mechanisms
from bgcellmodels.models.STN import Miocinovic2006 as miocinovic
from bgcellmodels.models.axon.mcintyre2002 import AxonMcintyre2002

## Settings

In [2]:
template = 'STN_morph_arcdist'
miocinovic.load_template(template)
template_constructor = getattr(h, template)

# Global variables
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Miocinovic (2006)

## Make Cell

In [3]:
# Instantiate template
icell = template_constructor()
icell.with_extracellular = 0

# Load morphology into template
morph_path = os.path.join(miocinovic.pkg_dir, 'morphologies/type1RD_axonless-with-AIS.swc')
morphology = ephys.morphologies.NrnFileMorphology(morph_path, do_replace_axon=False)
morphology.instantiate(sim=nrnsim, icell=icell)

# Setup biophysical properties
icell.del_unused_sections()
icell.insert_biophys()
nseg_extra = electrotonic.set_min_nseg_hines(icell.all, f_lambda=100.0)
print("Created {} extra segments to satisfy Hines' rule".format(nseg_extra))
icell.set_biophys_spatial()

One point section Import3d_Section[1] ending at line 2 has been removed
	and child Import3d_Section[24] reattached
Created 21 extra segments to satisfy Hines' rule
maximum dendrite length in STN_morph_arcdist[0]  is 857.16795 


0.0

In [4]:
# Load axon coordinates
streamline_path = '/home/luye/Documents/mri_data/Waxholm_rat_brain_atlas/WHS_DTI_v1_ALS/S56280_track_filter-ROI-STN.tck'
axon_builder = AxonMcintyre2002()
tracks_coords = morph_ni.load_streamlines(streamline_path, max_num=1, min_length=2.0)
axon_coords = tracks_coords[0]

# Build axon
axon_initial_secs = list(icell.axonal)
axon_terminal_secs = treeutils.leaf_sections(axon_initial_secs[0], subtree=True)
assert len(axon_terminal_secs) == 1
axon_end_sec = axon_terminal_secs[0]

axon = axon_builder.build_along_streamline(axon_coords,
            terminate='nodal_cutoff', interp_method='cartesian',
            parent_cell=icell, parent_sec=axon_end_sec,
            connection_method='translate_axon_start')

AssertionError: 

## Electrotonic Properties

## Stimulation Protocols

# Bevan, Chu (2016)

Can be found on neuromorpho.org using following searches
- [keyword 'subthalamic', sorty by archive](http://neuromorpho.org/KeywordBrowseView.jsp?count=52&keywords=%22subthalamic%22&browseBy=archive)
- [search by brain region 'basal ganglia'](http://neuromorpho.org/byregion.jsp#top)